In [3]:
!pip install cplex
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from docplex.cp.model import CpoModel
import numpy as np

# Function to get Multiplication Tensor $T_n$

In [6]:
# Implementation for square matrices
def multiplication_tensor(N=2):
    """Multiplication tensor.
    The multiplication tensor T of order N is defined by:
        C == A @ B <=> vec(C) == T x1 vec(A.T) x2 vec(B.T)
    where A, B, and C are N x N matrices and vec is the column-wise
    vectorization operator.
    """
    T = np.zeros((N ** 2, N ** 2, N ** 2), dtype=np.int64)
    for n in range(N):
        for m in range(N):
            u = np.ravel_multi_index(
                (n * np.ones(N, dtype=np.int64), np.arange(N)), (N, N))
            v = np.ravel_multi_index(
                (np.arange(N), m * np.ones(N, dtype=np.int64)), (N, N))
            w = np.ravel_multi_index(
                (m, n), (N, N)) * np.ones(len(u), dtype=np.int64)
            
            T[u, v, w] = 1
    # Assert cyclic symmetry.
    assert np.all(T == np.transpose(T, [2, 0, 1]))
    assert np.all(T == np.transpose(T, [1, 2, 0]))
    
    return T


In [7]:
# Implementation for square and non-square matrices
def general_multiplication_tensor(M, P, N):
   """Multiplication tensor.
   The multiplication tensor T in {0,1} of size MP x PN x MN
   for the multiplication of two matrices of size MxP and PxN
   """
   T = np.zeros((M * P, P * N, M * N), dtype=np.int64)

   for m in range(M):
       for p in range(P):
           for n in range(N):
               # Convert multi-dimensional indices to flat indices.
               a_index = np.ravel_multi_index((m, p), (M, P))
               b_index = np.ravel_multi_index((p, n), (P, N))
               c_index = np.ravel_multi_index((m, n), (M, N))
               T[a_index, b_index, c_index] = 1

   # Assert cyclic symmetry.
   #assert np.all(T == np.transpose(T, [2, 0, 1]))
   #assert np.all(T == np.transpose(T, [1, 2, 0]))

   return T


# Constraint Programming Formulations

### Simple square CP formulation

In [8]:
def CP(n,T,R):
    mdl = CpoModel()
    U = [[mdl.integer_var(-1, 1, name="U" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    V = [[mdl.integer_var(-1, 1, name="V" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    W = [[mdl.integer_var(-1, 1, name="W" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    for i in range(n**2):
        for j in range(n**2):
            for k in range(n**2):
                mdl.add(mdl.sum(U[i][r]*V[j][r]*W[k][r] for r in range(R)) == T[i][j][k])
    msol = mdl.solve()
    if msol:
      return [msol,U,V,W]
    else:
      print("Infeasible")
      return -1

## Cyclic Invariant square CP formulation

In [10]:
def Cp_opt_cyclic(n,T_n,S,R):
    T = int((R-S)/3)
    print(T)
  
    mdl = CpoModel()
    A = [[mdl.integer_var(-1, 1, name="A" + str(n) + "_" + str(r)) for r in range(S)] for n in range(n**2)]
    B = [[mdl.integer_var(-1, 1, name="B" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    C = [[mdl.integer_var(-1, 1, name="C" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    D = [[mdl.integer_var(-1, 1, name="D" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    
    for i in range(n**2):
        for j in range(n**2):
            for k in range(n**2):
              r_sum = []
              for r in range(R):
                if r <=S-1:
                 
                  r_sum.append(A[i][r]*A[j][r]*A[k][r])
                elif r>=S and r<=S-1+T:

                    r_sum.append(B[i][r-S]*D[j][r-S]*C[k][r-S])
                elif r>S-2+T and r<=S-1+2*T:
                     r_sum.append(C[i][r-S-T]*B[j][r-S-T]*D[k][r-S-T])
                else:
                  r_sum.append(D[i][r-S-2*T]*C[j][r-S-2*T]*B[k][r-S-2*T])

              mdl.add(mdl.sum(r_sum) == T_n[i][j][k])

              #mdl.add(mdl.sum(A[i][r]*A[i][r]*A[i][r] for r in range(S)) + mdl.sum(U[i][r]*V[j][r]*W[k][r] for r in range(R)) == T[i][j][k])
    msol = mdl.solve()
    if msol:
      return [msol,A,B,C,D]
    else:
      print("Infeasible")
      return -1


# Simple CP formulation for general matrix multiplication

In [17]:
def CP_general(M,P,N,T,R):
    mdl = CpoModel()
    U = [[mdl.integer_var(-1, 1, name="U" + str(n) + "_" + str(r)) for r in range(R)] for n in range(M*P)]
    V = [[mdl.integer_var(-1, 1, name="V" + str(n) + "_" + str(r)) for r in range(R)] for n in range(P*N)]
    W = [[mdl.integer_var(-1, 1, name="W" + str(n) + "_" + str(r)) for r in range(R)] for n in range(M*N)]
    for i in range(M*P):
        for j in range(P*N):
            for k in range(M*N):
                mdl.add(mdl.sum(U[i][r]*V[j][r]*W[k][r] for r in range(R)) == T[i][j][k])
    msol = mdl.solve()
    if msol:
      return [msol,U,V,W]
    else:
      print("Infeasible")
      return -1

# Function to retrieve $T_n$ given rank 1 decomposition matrices U,V,W

In [11]:
def expand_pd(U, V, W):
    """Expand a polyadic decomposition.
    The polyadic expansion T of the factor matrices U, V, and W is defined by:
        T[i, j, k] = \sum_r U[i, r] * V[j, r] * W[k, r].
    """
    I, J, K, R = U.shape[0], V.shape[0], W.shape[0], U.shape[1]
    T = np.zeros((I, J, K))
    for i in range(I):
        for j in range(J):
            for k in range(K):
                for r in range(R):
                    T[i, j, k] += U[i, r] * V[j, r] * W[k, r]
    return T

# Test our Implementations

#### Implementation 1 --> simplest CP formulation for square matrices

In [ ]:
N = 2
R = 7
T = multiplication_tensor(N)
solution = CP(N,T,R)

if type(solution)!=int:
  sol,U,V,W = solution
else:
  print("Infeasible")

U_sol = np.zeros((N**2,R))
V_sol = np.zeros((N**2,R))
W_sol = np.zeros((N**2,R))

for i in range(N**2):
  for r in range(R):
    U_sol[i,r] = sol[U[i][r]]
    V_sol[i,r] = sol[V[i][r]]
    W_sol[i,r] = sol[W[i][r]]

print("\n\n\n\n\n\n")
t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)
if (t_constraint_programming==T).all():
  print("Holy **** CP got the correct T_n")

#### Implementation 2 --> cyclic invariant CP formulation for square matrices

In [ ]:
# simples 2 by 2 square case --> rank = 7 for strassen's solution
# there are only two working S for 2 by 2 it is S = 1 and S = 4

N = 2
R = 7
S = 1
T = int((R-S)/3)
T_n = multiplication_tensor(N)
solution = Cp_opt_cyclic(N,T_n,S,R)

if type(solution)!=int:
  sol,A,B,C,D= solution
else:
  print("Infeasible")

A_sol = np.zeros((N**2,S))
B_sol = np.zeros((N**2,T))
C_sol = np.zeros((N**2,T))
D_sol = np.zeros((N**2,T))

for i in range(N**2):
  for r in range(S):
    A_sol[i,r] = sol[A[i][r]]
  for r in range(T):
    B_sol[i,r] = sol[B[i][r]]
    C_sol[i,r] = sol[C[i][r]]
    D_sol[i,r] = sol[D[i][r]]


U_sol = np.hstack((A_sol, B_sol, C_sol, D_sol))
V_sol = np.hstack((A_sol, D_sol, B_sol, C_sol))
W_sol = np.hstack((A_sol, C_sol, D_sol, B_sol))
print("\n\n\n\n\n\n")
t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)
if (t_constraint_programming==T_n).all():
  print("Holy **** CP got the correct T_n using cyclic invariance constraints")

# Now let us check our implementations for non-square matrix multiplications

In [19]:
M = 2
P = 3
N = 2
T_n = general_multiplication_tensor(M, P, N)

R = 11

solution = CP_general(M,P,N,T_n,R)

if type(solution)!=int:
  sol,U,V,W = solution
else:
  print("Infeasible")

U_sol = np.zeros((M*P,R))
V_sol = np.zeros((P*N,R))
W_sol = np.zeros((M*N,R))

for i in range(M*P):
  for r in range(R):
    U_sol[i,r] = sol[U[i][r]]
for i in range(N*P):
  for r in range(R):
    V_sol[i,r] = sol[V[i][r]]
for i in range(M*N):
  for r in range(R):
    W_sol[i,r] = sol[W[i][r]]

print("\n\n\n\n\n\n")
t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)
if (t_constraint_programming==T_n).all():
  print("Holy **** CP got the correct T_n for non square matrices")

Streaming output truncated to the last 5000 lines.
                    1779k         35    1   F     1  = V4_2
                    1786k         62    2   F     1  = V2_10
                    1780k         68    1         0  = U2_6
                    1787k         40    2   F     1  = V1_8
                    1781k         45    1         0  = V2_2
                    1788k         44    2   F    -1 != W1_10
                    1789k         41    2         0  = U3_9
                    1782k         10    1        -1 != V0_7
                    1790k         41    2         1 != V2_8
                    1783k         62    1   F     1  = U2_6
                    1791k         62    2        -1  = U1_5
                    1784k         57    1   F    -1  = W0_8
                    1792k         61    2        -1 != W0_4
                    1785k         44    1         0 != U0_4
                    1793k         43    2   F     0  = V0_3
                    1786k         59    1   F  

# Implementing Symmetry handling per overleaf doc made by Elias

In [ ]:
def CP_opt_symmetry(n,T,R):
    mdl = CpoModel()
    U = [[mdl.integer_var(-1, 1, name="U" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    V = [[mdl.integer_var(-1, 1, name="V" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    W = [[mdl.integer_var(-1, 1, name="W" + str(n) + "_" + str(r)) for r in range(R)] for n in range(n**2)]
    for i in range(n**2):
        for j in range(n**2):
            for k in range(n**2):
                mdl.add(mdl.sum(U[i][r]*V[j][r]*W[k][r] for r in range(R)) == T[i][j][k])

    # Cardinality ordering constraint

    for r in range(R-1):
      mdl.add(
          mdl.sum(mdl.abs(U[i][r]) for i in range(len(U))) + mdl.sum(mdl.abs(V[j][r]) for j in range(len(V)))<=
          mdl.sum(mdl.abs(U[i][r+1]) for i in range(len(U))) + mdl.sum(mdl.abs(V[j][r+1]) for j in range(len(V)))
          )
    
    msol = mdl.solve()
    if msol:
      return [msol,U,V,W]
    else:
      print("Infeasible")
      return -1